In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
# Custom function to build hidden combination (i.e. convolution) layers

def create_random_combination_layer(input_layer, combination_size, num_combinations, input_dim):
    outputs = []
    
    for _ in range(num_combinations):
        # First random feature selection
        indices_1 = np.random.choice(input_dim, combination_size, replace=False)
        indices_tensor_1 = tf.constant(indices_1, dtype=tf.int32)
        
        # First feature selection using Lambda layer
        slice_layer_1 = Lambda(
            lambda x: tf.gather(x, indices_tensor_1, axis=1),  # Gather selected features
            output_shape=(combination_size,)
        )(input_layer)
        
        # Second random feature selection (after the first random selection)
        indices_2 = np.random.choice(combination_size, combination_size, replace=False)
        indices_tensor_2 = tf.constant(indices_2, dtype=tf.int32)
        
        # Second feature selection using Lambda layer
        slice_layer_2 = Lambda(
            lambda x: tf.gather(x, indices_tensor_2, axis=1),  # Apply a second feature selection
            output_shape=(combination_size,)
        )(slice_layer_1)

        # Apply Dense layers on the final selected subset
        selected_features = Dense(16, activation='relu')(
            Dense(8, activation='relu')(
                Dense(4, activation='relu')(slice_layer_2)
            )
        )
        outputs.append(selected_features)
    
    # Concatenate the outputs from all the random feature combinations
    return Concatenate()(outputs)

In [5]:
def weighted_categorical_crossentropy(weights):
    # Convert the dictionary to a list or array of weights
    weights_array = np.array([weights[key] for key in sorted(weights.keys())])

    def loss(y_true, y_pred):
        y_true = K.cast(y_true, 'float32')
        weights_tensor = K.constant(weights_array)  # Use the weights as a tensor
        weights_applied = K.sum(y_true * weights_tensor, axis=-1)  # Apply weights to true labels
        return K.categorical_crossentropy(y_true, y_pred) * weights_applied

    return loss

In [6]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Perform a stratified split into train and validation sets (80% train, 20% validation)
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y)
    del train_X
    del train_y
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # Parameters for Random Feature Combination
    num_combinations = 20  # Number of random column combinations
    combination_size = 3   # Number of columns in each combination

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 5

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):

        # Compute class weights for training data
        class_weights = compute_class_weight(
            class_weight='balanced',
            classes=np.unique(y_train.argmax(axis=1)),  # Convert one-hot back to class indices
            y=y_train.argmax(axis=1)
        )

        # Convert class weights to a dictionary
        class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        input_layer = Input(shape=(X_train.shape[1],))  # Input shape from the training data
        feature_layer = create_random_combination_layer(input_layer, combination_size, num_combinations, X_train.shape[1])
        hidden_layer = Dense(128, activation='relu')(feature_layer)
        hidden_layer = Dropout(0.5)(hidden_layer)
        output_layer = Dense(test_y.shape[1], activation='softmax')(hidden_layer)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss=weighted_categorical_crossentropy(class_weights_dict), metrics=['accuracy'])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            X_train, y_train, 
            epochs=1000, 
            batch_size=int(X_train.shape[0] * 0.01), 
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

    # Average the metrics over all runs
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [7]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 1.4193
test_accuracy: 0.8111
balanced_accuracy: 0.6652
roc_auc: 0.9025
pr_auc: 0.9534
f2: 0.8272
mcc: 0.6684
0_precision: 0.9903
0_recall: 0.7955
0_f1-score: 0.8751
0_support: 9117.0000
1_precision: 0.3672
1_recall: 0.5600
1_f1-score: 0.2658
1_support: 10.0000
2_precision: 0.1629
2_recall: 0.4529
2_f1-score: 0.1943
2_support: 34.0000
3_precision: 0.7393
3_recall: 0.8405
3_f1-score: 0.7726
3_support: 1781.0000
4_precision: 0.9311
4_recall: 0.9740
4_f1-score: 0.9497
4_support: 653.0000
5_precision: 0.1806
5_recall: 0.5000
5_f1-score: 0.2487
5_support: 2.0000
6_precision: 0.0606
6_recall: 0.5333
6_f1-score: 0.0935
6_support: 3.0000
accuracy: 0.8111
macro avg_precision: 0.4903
macro avg_recall: 0.6652
macro avg_f1-score: 0.4857
macro avg_support: 11600.0000
weighted avg_precision: 0.9451
weighted avg_recall: 0.8111
weighted avg_f1-score: 0.8607
weighted avg_support

In [8]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 1.8155
test_accuracy: 0.1248
balanced_accuracy: 0.2213
roc_auc: 0.5882
pr_auc: 0.3987
f2: 0.1032
mcc: 0.0555
0_precision: 0.2794
0_recall: 0.1135
0_f1-score: 0.1222
0_support: 42368.0000
1_precision: 0.7418
1_recall: 0.0729
1_f1-score: 0.1104
1_support: 56661.0000
2_precision: 0.3185
2_recall: 0.5098
2_f1-score: 0.1789
2_support: 7151.0000
3_precision: 0.0626
3_recall: 0.3413
3_f1-score: 0.0311
3_support: 549.0000
4_precision: 0.0181
4_recall: 0.1057
4_f1-score: 0.0291
4_support: 1899.0000
5_precision: 0.0174
5_recall: 0.2252
5_f1-score: 0.0301
5_support: 3473.0000
6_precision: 0.0259
6_recall: 0.1809
6_f1-score: 0.0435
6_support: 4102.0000
accuracy: 0.1248
macro avg_precision: 0.2091
macro avg_recall: 0.2213
macro avg_f1-score: 0.0779
macro avg_support: 116203.0000
weighted avg_precision: 0.4852
weighted avg_recall: 0.1248
weighted avg_f1-score: 0.1125
weighte

In [9]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("/kaggle/input/ma-datasets/kdd_train.csv", "/kaggle/input/ma-datasets/kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 3.3536
test_accuracy: 0.0442
balanced_accuracy: 0.1268
roc_auc: 0.6113
pr_auc: 0.9289
f2: 0.0468
mcc: 0.0676
0_precision: 0.3917
0_recall: 0.0359
0_f1-score: 0.0611
0_support: 194557.0000
1_precision: 0.1848
1_recall: 0.1733
1_f1-score: 0.1788
1_support: 3178.0000
2_precision: 0.0000
2_recall: 0.0000
2_f1-score: 0.0000
2_support: 2496.0000
3_precision: 0.3612
3_recall: 0.5322
3_f1-score: 0.2310
3_support: 2083.0000
4_precision: 0.0414
4_recall: 0.0834
4_f1-score: 0.0553
4_support: 463.0000
5_precision: 0.0028
5_recall: 0.4952
5_f1-score: 0.0055
5_support: 441.0000
6_precision: 0.0154
6_recall: 0.0598
6_f1-score: 0.0245
6_support: 204.0000
7_precision: 0.0019
7_recall: 0.3663
7_f1-score: 0.0038
7_support: 196.0000
8_precision: 0.0023
8_recall: 0.3358
8_f1-score: 0.0046
8_support: 53.0000
9_precision: 0.0044
9_recall: 0.1818
9_f1-score: 0.0086
9_support: 11.0000


In [10]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 1.6978
test_accuracy: 0.1870
balanced_accuracy: 0.3736
roc_auc: 0.7316
pr_auc: 0.8229
f2: 0.1885
mcc: 0.1234
0_precision: 0.9733
0_recall: 0.1578
0_f1-score: 0.2433
0_support: 26862.0000
1_precision: 0.3052
1_recall: 0.2253
1_f1-score: 0.2451
1_support: 2657.0000
2_precision: 0.1813
2_recall: 0.8742
2_f1-score: 0.2666
2_support: 908.0000
3_precision: 0.0650
3_recall: 0.0904
3_f1-score: 0.0673
3_support: 522.0000
4_precision: 0.0137
4_recall: 0.4000
4_f1-score: 0.0241
4_support: 293.0000
5_precision: 0.0203
5_recall: 0.2134
5_f1-score: 0.0339
5_support: 269.0000
6_precision: 0.0954
6_recall: 0.1793
6_f1-score: 0.0597
6_support: 116.0000
7_precision: 0.0353
7_recall: 0.5585
7_f1-score: 0.0633
7_support: 53.0000
8_precision: 0.0507
8_recall: 0.6636
8_f1-score: 0.0901
8_support: 44.0000
accuracy: 0.1870
macro avg_precision: 0.1934
macro avg_recall: 0.3736
macro avg